nama: M. Salisianto Fahmi Zaka <br>
nim : 190411100027 <br>
matakuliah : ppw kelas A

## Install & Import Library
Jika anda ingin menjalankan notebook secara offline seperti Jupyter Notebook, pastikan perangkat anda sudah terinstall library yang dibutuhkan. Jika anda ingin menjalankan notebook secara online seperti Google Colaboratory, pastikan notebook tersebut sudah terinstall library yang dibutuhkan. Library yang dibutuhkan dalam proyek ini, yaitu:
- Scrapy
- OS
- Regex
- Pandas
- Matplotlib

In [ ]:
!pip install Sastrawi
!pip install Scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import Library
import os
import regex as re
import pandas as pd
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# Install NLTK Corpus
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Crawling Data

## Create Scrapy Project

Pada bagian ini digunakan untuk membuat proyek library Scrapy dan memindah posisi direktori. Proyek library Scrapy diberi nama crawlproject. Posisi direktori dipindah ke crawlproject/crawlproject/spiders

In [ ]:
# Membuat proyek library Scrapy
!scrapy startproject crawlproject

Error: scrapy.cfg already exists in /content/crawlproject


In [ ]:
# Melihat posisi direktori saat ini
os.getcwd()

'/content'

In [ ]:
# Mengubah posisi direktori saat ini ke crawlproject/crawlproject/spiders
# Fungsinya agar bisa menjalankan file proyek library Scrapy
os.chdir('crawlproject/crawlproject/spiders')
os.getcwd()

'/content/crawlproject/crawlproject/spiders'

## Crawling Link PTA

Pada bagian ini digunakan untuk membuat dan menjalankan program python. Program tersebut digunakan untuk melakukan _crawling_ 60 link . Untuk melakukan _crawling_ menggunakan library scrapy.

In [ ]:
%%writefile -a link.py
# Membuat file link.py
# File link.py digunakan untuk crawling link tugas akhir
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        start_urls = ['https://pta.trunojoyo.ac.id/c_search/byprod/10/1']
        for i in range (2,13):
            tambah = 'https://pta.trunojoyo.ac.id/c_search/byprod/10/'+ str(i)
            start_urls.append(tambah)
        for url in start_urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for i in range(1, 6):
            yield {
                'link':response.css('#content_journal > ul > li:nth-child(' +str(i)+ ') > div:nth-child(3) > a::attr(href)').extract()
            }

Appending to link.py


In [ ]:
# Menjalankan file link.py untuk melakukan proses crawling link tugas akhir
# Hasil akan disimpan dalam file link.csv
# File link.csv digunakan untuk melakukan crawling detail tugas akhir
!scrapy runspider link.py -o link.csv

/usr/local/lib/python3.7/dist-packages/scrapy/spiderloader.py:40: UserWarning: There are several spiders with the same name:

  QuotesSpider named 'quotes' (in crawlproject.spiders.detail)

  QuotesSpider named 'quotes' (in crawlproject.spiders.link)

  This can cause unexpected behavior.
  category=UserWarning,
2022-09-06 15:38:30 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: crawlproject)
2022-09-06 15:38:30 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-09-06 15:38:30 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawlproject',
 'NEWSPIDER_MODULE': 'crawlproject.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'SPIDER_MODULES': ['crawlproject.spiders']}
2022-09-06 15:38:30 [scrapy.u

## Crawling Detail PTA

Pada bagian ini digunakan untuk membuat dan menjalankan program python. Program tersebut digunakan untuk melakukan crawling 60n. Untuk melakukan crawling menggunakan library scrapy.

In [ ]:
%%writefile -a detail.py
# Membuat file detail.py
# File detail.py digunakan untuk crawling detail tugas akhir
import scrapy
import pandas as pd

class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        dataCSV = pd.read_csv('link.csv')
        indexData = dataCSV.iloc[:, [0]].values
        arrayData = []
        for i in indexData:
            ambil = i[0]
            arrayData.append(ambil)
        for url in arrayData:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        yield {
            'judul': response.css('#content_journal > ul > li > div:nth-child(2) > a::text').extract(),
            'penulis': response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(2) > span::text').extract(),
            'pembimbing_1': response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(3) > span::text').extract(),
            'pembimbing_2': response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(4) > span::text').extract(),
            'abstrak': response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(2) > p::text').extract()
        }

Appending to detail.py


In [ ]:
# Menjalankan file detail.py untuk melakukan proses crawling detail tugas akhir
# Hasil akan disimpan dalam file detail.csv
# File detail.csv digunakan sebagai dataset utama yang diolah dalam proyek ini
!scrapy runspider detail.py -o detail.csv

/usr/local/lib/python3.7/dist-packages/scrapy/spiderloader.py:40: UserWarning: There are several spiders with the same name:

  QuotesSpider named 'quotes' (in crawlproject.spiders.detail)

  QuotesSpider named 'quotes' (in crawlproject.spiders.link)

  This can cause unexpected behavior.
  category=UserWarning,
2022-09-06 15:38:38 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: crawlproject)
2022-09-06 15:38:38 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-09-06 15:38:38 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawlproject',
 'NEWSPIDER_MODULE': 'crawlproject.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'SPIDER_MODULES': ['crawlproject.spiders']}
2022-09-06 15:38:39 [scrapy.u